# Gensim Word2Vec Tutorial
This notebook is modified based on the original one provided by Gensim.  
You can find it here: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/word2vec.ipynb

Using large amounts of **unannotated plain text**, word2vec learns relationships between words automatically.  
The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like:  
  
**vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”)**  
**vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.**  
  
Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.  
Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. 
That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.  

## Preparing the Input & quick start
gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings)

In [1]:
# for better logging information
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentence1 = ['first', 'sentence']
sentence2 = ['second', 'sentence']
sentences = [sentence1, sentence2]

train word2vec on the two sentences

In [3]:
model = gensim.models.Word2Vec(sentences, min_count=1)

2018-01-10 17:09:11,119 : INFO : collecting all words and their counts
2018-01-10 17:09:11,120 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-10 17:09:11,121 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2018-01-10 17:09:11,121 : INFO : Loading a fresh vocabulary
2018-01-10 17:09:11,122 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-01-10 17:09:11,123 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-01-10 17:09:11,123 : INFO : deleting the raw counts dictionary of 3 items
2018-01-10 17:09:11,124 : INFO : sample=0.001 downsamples 3 most-common words
2018-01-10 17:09:11,125 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-01-10 17:09:11,125 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-01-10 17:09:11,210 : INFO : resetting layer weights
2018-01-10 17:09:11,211 : INFO : training model with 3 workers o

**gensim.models.Word2Vec** is the same as **gensim.models.word2vec.Word2Vec**

In [4]:
gensim.models.Word2Vec

gensim.models.word2vec.Word2Vec

### Gensim only requires that the input must provide sentences sequentially, when iterated over.

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.  
**No need to keep everything in RAM**: we can provide one sentence, process it, forget it, load another sentence.  
For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

create some toy data to use with the following example:  
**generate sentences from a folder**

In [5]:
import smart_open, os

if not os.path.exists('./data_toy/'):
    os.makedirs('./data_toy/')
    
filenames = ['./data_toy/f1.txt', './data_toy/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [6]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
    
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [7]:
sentences = MySentences('./data_toy/')

In [8]:
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


get **word vectors** from the trained model

In [9]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.keyedvectors.Vocab object at 0x7f87a8745710>, 'first': <gensim.models.keyedvectors.Vocab object at 0x7f87710116d0>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7f8771011690>}


**Say we want to further preprocess the words from the files** — convert to unicode, lowercase, remove numbers, extract named entities and so on.  
All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.  

### Note to advanced users:
calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator.  
In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language.  
  
1. The first pass collects words and their frequencies to build an internal dictionary tree structure.  
2. The second pass trains the neural model.  

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [10]:
# first build an empty model, no training
new_model = gensim.models.Word2Vec(min_count=1)  
new_model.build_vocab(sentences)                     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)

2018-01-10 17:11:59,320 : INFO : collecting all words and their counts
2018-01-10 17:11:59,323 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-10 17:11:59,325 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2018-01-10 17:11:59,327 : INFO : Loading a fresh vocabulary
2018-01-10 17:11:59,329 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-01-10 17:11:59,330 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-01-10 17:11:59,332 : INFO : deleting the raw counts dictionary of 3 items
2018-01-10 17:11:59,334 : INFO : sample=0.001 downsamples 3 most-common words
2018-01-10 17:11:59,335 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-01-10 17:11:59,337 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-01-10 17:11:59,338 : INFO : resetting layer weights
2018-01-10 17:11:59,340 : INFO : training model with 3 workers o

0

### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim)

In [11]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'
print lee_train_file

/home/saucecat/anaconda2/lib/python2.7/site-packages/gensim/test/test_data/lee_background.cor


In [12]:
class MySentencesFromFile(object):
    def __init__(self, fname):
        self.fname = fname
    
    def __iter__(self):
        for line in open(self.fname):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

In [13]:
sentences = MySentencesFromFile(lee_train_file)

## Training Word2Vec
**Word2Vec** accepts several parameters that affect both training speed and quality.

### min_count
**min_count** is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them.

In [14]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2018-01-10 17:12:20,911 : INFO : collecting all words and their counts
2018-01-10 17:12:20,985 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-10 17:12:21,043 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-01-10 17:12:21,044 : INFO : Loading a fresh vocabulary
2018-01-10 17:12:21,050 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2018-01-10 17:12:21,051 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2018-01-10 17:12:21,054 : INFO : deleting the raw counts dictionary of 10186 items
2018-01-10 17:12:21,055 : INFO : sample=0.001 downsamples 54 most-common words
2018-01-10 17:12:21,055 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2018-01-10 17:12:21,056 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2018-01-10 17:12:21,059 : INFO : resetting layer weights
2018-01-10 17:1

### size
**size** is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.  
Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [15]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2018-01-10 17:12:22,450 : INFO : collecting all words and their counts
2018-01-10 17:12:22,453 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-10 17:12:22,493 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-01-10 17:12:22,494 : INFO : Loading a fresh vocabulary
2018-01-10 17:12:22,500 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2018-01-10 17:12:22,501 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2018-01-10 17:12:22,507 : INFO : deleting the raw counts dictionary of 10186 items
2018-01-10 17:12:22,508 : INFO : sample=0.001 downsamples 49 most-common words
2018-01-10 17:12:22,510 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2018-01-10 17:12:22,510 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2018-01-10 17:12:22,515 : INFO : resetting layer weights
2018-01-10 17:

### workers
**workers**, the last of the major parameters is for training parallelization, to speed up training.

In [16]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2018-01-10 17:12:24,627 : INFO : collecting all words and their counts
2018-01-10 17:12:24,628 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-10 17:12:24,642 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-01-10 17:12:24,643 : INFO : Loading a fresh vocabulary
2018-01-10 17:12:24,651 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2018-01-10 17:12:24,652 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2018-01-10 17:12:24,657 : INFO : deleting the raw counts dictionary of 10186 items
2018-01-10 17:12:24,658 : INFO : sample=0.001 downsamples 49 most-common words
2018-01-10 17:12:24,658 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2018-01-10 17:12:24,659 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2018-01-10 17:12:24,663 : INFO : resetting layer weights
2018-01-10 17:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).  
You can check the number of cores in your computer by:

In [17]:
import multiprocessing

cores = multiprocessing.cpu_count()
print(cores)
assert gensim.models.word2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

8


## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **vocabulary** (controlled by min_count parameter) times **size** (size parameter) of floats (single precision aka 4 bytes).  
Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.  
There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.  

### syntactic and semantic test
**Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.**  
For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.  
The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

**Gensim supports the same evaluation set, in exactly the same format**  
questions-words.txt can be downloaded here: https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt

In [19]:
accuracy_result = model.accuracy('../../gensim_word2vec/data/questions-words.txt')

2018-01-10 17:15:15,643 : INFO : family: 0.0% (0/2)
2018-01-10 17:15:15,677 : INFO : gram3-comparative: 0.0% (0/12)
2018-01-10 17:15:15,700 : INFO : gram4-superlative: 0.0% (0/12)
2018-01-10 17:15:15,763 : INFO : gram5-present-participle: 5.0% (1/20)
2018-01-10 17:15:15,800 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2018-01-10 17:15:15,826 : INFO : gram7-past-tense: 0.0% (0/20)
2018-01-10 17:15:15,843 : INFO : gram8-plural: 0.0% (0/12)
2018-01-10 17:15:15,852 : INFO : total: 1.0% (1/98)


The accuracy text actually contains 14 sections.

In [25]:
for r in accuracy_result:
    print r['section']

capital-common-countries
capital-world
currency
city-in-state
family
gram1-adjective-to-adverb
gram2-opposite
gram3-comparative
gram4-superlative
gram5-present-participle
gram6-nationality-adjective
gram7-past-tense
gram8-plural
gram9-plural-verbs
total


This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



### evaluate semantic similarity
In the December 2016 release of Gensim we added a better way to **evaluate semantic similarity**.  
By default it uses an academic dataset **WS-353** but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. **It measures the relatedness or co-occurrence of two words.**  
For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [26]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

/home/saucecat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `evaluate_word_pairs` (Method will be removed in 4.0.0, use self.wv.evaluate_word_pairs() instead).
  """Entry point for launching an IPython kernel.
2018-01-10 17:18:38,622 : INFO : Pearson correlation coefficient against /home/saucecat/anaconda2/lib/python2.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0959
2018-01-10 17:18:38,624 : INFO : Spearman rank-order correlation coefficient against /home/saucecat/anaconda2/lib/python2.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0772
2018-01-10 17:18:38,626 : INFO : Pairs with unknown words ratio: 85.6%


((0.095890244799869642, 0.5032649041421664),
 SpearmanrResult(correlation=0.077203303718994309, pvalue=0.59024692601285134),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [27]:
from tempfile import mkstemp

# creates a temp file
fs, temp_path = mkstemp("gensim_temp")  

# save the model
model.save(temp_path)  

2018-01-10 17:18:54,752 : INFO : saving Word2Vec object under /tmp/tmpDkkx3Dgensim_temp, separately None
2018-01-10 17:18:54,754 : INFO : not storing attribute syn0norm
2018-01-10 17:18:54,757 : INFO : not storing attribute cum_table
2018-01-10 17:18:54,784 : INFO : saved /tmp/tmpDkkx3Dgensim_temp


In [28]:
# open the model
new_model = gensim.models.Word2Vec.load(temp_path)  

2018-01-10 17:19:01,502 : INFO : loading Word2Vec object from /tmp/tmpDkkx3Dgensim_temp
2018-01-10 17:19:01,507 : INFO : loading wv recursively from /tmp/tmpDkkx3Dgensim_temp.wv.* with mmap=None
2018-01-10 17:19:01,507 : INFO : setting ignored attribute syn0norm to None
2018-01-10 17:19:01,508 : INFO : setting ignored attribute cum_table to None
2018-01-10 17:19:01,509 : INFO : loaded /tmp/tmpDkkx3Dgensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.  
In addition, you can load models created by the original C tool, both using its text and binary formats:  

```python
model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
# using gzipped/bz2 input works too, no need to unzip:
model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [29]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2018-01-10 17:19:32,916 : INFO : loading Word2Vec object from /tmp/tmpDkkx3Dgensim_temp
2018-01-10 17:19:32,924 : INFO : loading wv recursively from /tmp/tmpDkkx3Dgensim_temp.wv.* with mmap=None
2018-01-10 17:19:32,925 : INFO : setting ignored attribute syn0norm to None
2018-01-10 17:19:32,925 : INFO : setting ignored attribute cum_table to None
2018-01-10 17:19:32,926 : INFO : loaded /tmp/tmpDkkx3Dgensim_temp
2018-01-10 17:19:32,929 : INFO : collecting all words and their counts
2018-01-10 17:19:32,930 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-10 17:19:32,931 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2018-01-10 17:19:32,931 : INFO : Updating model with new vocabulary
2018-01-10 17:19:32,933 : INFO : New added 0 unique words (0% of original 13) and increased the count of 0 pre-existing words (0% of original 13)
2018-01-10 17:19:32,933 : INFO : deleting the raw counts dictionary of 13 items
2018-01-10 17:19:32

You may need to tweak the **total_words** parameter to `train()`, depending on what **learning rate decay** you want to simulate.  
Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.  

## Using the model
**Word2Vec** supports several word similarity tasks out of the box:

In [32]:
model.wv.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

[('longer', 0.9883442521095276)]

In [33]:
model.wv.doesnt_match("input is lunch he sentence cat".split())

2018-01-10 17:20:08,053 : WARNING : vectors for words set(['lunch', 'input', 'cat']) are not present in the model, ignoring these words


'sentence'

In [34]:
print(model.wv.similarity('human', 'party'))
print(model.wv.similarity('tree', 'murder'))

0.999158114566
0.995601177348


You can get the probability distribution for the center word given the context words as input:

In [36]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('more', 0.0010526252), ('continue', 0.00091675884), ('can', 0.0009019851), ('training', 0.00089129218), ('it', 0.00077612454), ('australia', 0.0007580528), ('there', 0.00074004085), ('could', 0.00073954405), ('or', 0.00073881348), ('government', 0.00073402957)]


The results here don't look good because the training corpus is very small.  
**To get meaningful results one needs to train on 500k+ words.**

### get word vector by word
If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [40]:
model.wv['tree']  # raw NumPy vector of a word

array([ 0.00521669,  0.02261025, -0.02910671, -0.00819009, -0.03401158,
       -0.03982855,  0.00848157, -0.09348959, -0.0145504 , -0.02479113,
        0.02644364, -0.02420322, -0.06826016, -0.02135213, -0.02130898,
        0.04832803, -0.02974672, -0.00157361,  0.02257499, -0.02790363,
       -0.06846678, -0.00610655,  0.07050072,  0.05158021,  0.02366703,
        0.01493037, -0.00511777,  0.01834179, -0.01667378, -0.00014777,
        0.00044038,  0.07425427,  0.00874526, -0.0033846 ,  0.02567542,
        0.02162404, -0.04740742,  0.01675648,  0.0124343 , -0.07041643,
       -0.08084929,  0.00612027, -0.04462328, -0.03064694,  0.03128598,
       -0.05806697,  0.04003579,  0.00400401, -0.0150517 ,  0.04091452,
        0.01718655,  0.00683087, -0.03807062,  0.0065935 ,  0.01423975,
       -0.02037616,  0.05841972, -0.02892845,  0.01907358, -0.03592079,
       -0.02926338, -0.05061468,  0.01876437, -0.03062048, -0.01111972,
       -0.06736362, -0.08027675,  0.03937999,  0.05804415, -0.00

…or en-masse as a 2D NumPy matrix from **model.wv.syn0**

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [41]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(sentences, min_count=1, compute_loss=True, hs=0, sg=1, seed=42)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

2018-01-10 17:22:12,526 : INFO : collecting all words and their counts
2018-01-10 17:22:12,527 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-10 17:22:12,542 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-01-10 17:22:12,543 : INFO : Loading a fresh vocabulary
2018-01-10 17:22:12,566 : INFO : min_count=1 retains 10186 unique words (100% of original 10186, drops 0)
2018-01-10 17:22:12,567 : INFO : min_count=1 leaves 59890 word corpus (100% of original 59890, drops 0)
2018-01-10 17:22:12,593 : INFO : deleting the raw counts dictionary of 10186 items
2018-01-10 17:22:12,594 : INFO : sample=0.001 downsamples 37 most-common words
2018-01-10 17:22:12,595 : INFO : downsampling leaves estimated 47231 word corpus (78.9% of prior 59890)
2018-01-10 17:22:12,595 : INFO : estimated required memory for 10186 words and 100 dimensions: 13241800 bytes
2018-01-10 17:22:12,619 : INFO : resetting layer weights
2018-01-10 17:22

1535374.75


### Benchmarks to see effect of training loss compuation code on training time

We first download and setup the test data used for getting the benchmarks.

In [42]:
input_data_files = []

def setup_input_data():
    # check if test data already present
    if os.path.isfile('./text8') is False:

        # download and decompress 'text8' corpus
        import zipfile
        ! wget 'http://mattmahoney.net/dc/text8.zip'
        ! unzip 'text8.zip'
    
    # create 1 MB, 10 MB and 50 MB files
    ! head -c1000000 text8 > text8_1000000
    ! head -c10000000 text8 > text8_10000000
    ! head -c50000000 text8 > text8_50000000
                
    # add 25 KB test file
    input_data_files.append(lee_train_file)

    # add 1 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_1000000'))

    # add 10 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_10000000'))

    # add 50 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_50000000'))

    # add 100 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8'))

setup_input_data()
print(input_data_files)

['/home/saucecat/anaconda2/lib/python2.7/site-packages/gensim/test/test_data/lee_background.cor', '/home/saucecat/Desktop/DL/NLP-playground/word_embedding/text8_1000000', '/home/saucecat/Desktop/DL/NLP-playground/word_embedding/text8_10000000', '/home/saucecat/Desktop/DL/NLP-playground/word_embedding/text8_50000000', '/home/saucecat/Desktop/DL/NLP-playground/word_embedding/text8']


We now compare the training time taken for different combinations of input data and model training parameters like `hs` and `sg`.

In [43]:
logging.getLogger().setLevel(logging.ERROR)

In [45]:
# using 25 KB and 50 MB files only for generating o/p -> comment next line for using all 5 test files
input_data_files = [input_data_files[0], input_data_files[-2]]
print(input_data_files)

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

for data_file in input_data_files:
    data = gensim.models.word2vec.LineSentence(data_file) 
    for sg_val in sg_values:
        for hs_val in hs_values:
            for loss_flag in [True, False]:
                time_taken_list = []
                for i in range(3):
                    start_time = time.time()
                    w2v_model = gensim.models.Word2Vec(data, compute_loss=loss_flag, sg=sg_val, hs=hs_val, seed=seed_val) 
                    time_taken_list.append(time.time() - start_time)

                time_taken_list = np.array(time_taken_list)
                time_mean = np.mean(time_taken_list)
                time_std = np.std(time_taken_list)
                train_time_values.append({'train_data': data_file, 'compute_loss': loss_flag, 'sg': sg_val, 'hs': hs_val, 'mean': time_mean, 'std': time_std})

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(by=['train_data', 'sg', 'hs', 'compute_loss'], ascending=[False, False, True, False])
print(train_times_table)

['/home/saucecat/anaconda2/lib/python2.7/site-packages/gensim/test/test_data/lee_background.cor', '/home/saucecat/anaconda2/lib/python2.7/site-packages/gensim/test/test_data/lee_background.cor']
    compute_loss  hs      mean  sg       std  \
4           True   0  0.386239   1  0.002996   
12          True   0  0.423641   1  0.003817   
5          False   0  0.392958   1  0.015561   
13         False   0  0.422403   1  0.003255   
6           True   1  0.791827   1  0.014086   
14          True   1  0.817583   1  0.001846   
7          False   1  0.866585   1  0.002731   
15         False   1  0.809655   1  0.002575   
0           True   0  0.168576   0  0.002199   
8           True   0  0.166439   0  0.002085   
1          False   0  0.164159   0  0.002096   
9          False   0  0.163657   0  0.001256   
2           True   1  0.287650   0  0.008225   
10          True   1  0.288234   0  0.000582   
3          False   1  0.274050   0  0.002848   
11         False   1  0.294971   0  0

### Adding Word2Vec "model to dict" method to production pipeline
Suppose, we still want more performance improvement in production. 
One good way is to cache all the similar words in a dictionary.
So that next time when we get the similar query word, we'll search it first in the dict.
And if it's a hit then we will show the result directly from the dictionary.
otherwise we will query the word and then cache it so that it doesn't miss next time.

In [47]:
most_similars_precalc = {word : model.wv.most_similar(word) for word in model.wv.index2word}
for i, (key, value) in enumerate(most_similars_precalc.iteritems()):
    if i==3:
        break
    print key, value

four [('over', 0.9999379515647888), ('local', 0.9999368786811829), ('after', 0.9999325275421143), ('on', 0.9999321699142456), ('world', 0.9999314546585083), ('last', 0.9999311566352844), ('and', 0.9999299645423889), ('from', 0.9999293088912964), ('into', 0.99992835521698), ('its', 0.9999280571937561)]
jihad [('gaza', 0.9995455741882324), ('and', 0.9995432496070862), ('on', 0.9995413422584534), ('palestinian', 0.9995368719100952), ('in', 0.9995279312133789), ('security', 0.9995211958885193), ('were', 0.9995205998420715), ('from', 0.999518632888794), ('former', 0.9995176196098328), ('other', 0.9995072484016418)]
captain [('are', 0.9998456835746765), ('out', 0.9998337626457214), ('over', 0.9998326897621155), ('any', 0.9998325705528259), ('now', 0.999830961227417), ('have', 0.9998291730880737), ('is', 0.9998286962509155), ('australian', 0.9998236298561096), ('at', 0.9998235106468201), ('up', 0.9998231530189514)]


### Comparison with and without caching

for time being lets take 4 words randomly

In [48]:
import time
words = ['voted','few','their','around']

Without caching

In [49]:
start = time.time()
for word in words:
    result = model.wv.most_similar(word)
    print(result)
end = time.time()
print(end-start)

[('action', 0.9987000226974487), ('would', 0.9986580610275269), ('could', 0.9986568093299866), ('need', 0.9986546635627747), ('will', 0.9986473917961121), ('it', 0.9986432790756226), ('legal', 0.9986417293548584), ('expected', 0.9986345171928406), ('there', 0.9986207485198975), ('we', 0.998618483543396)]
[('which', 0.9997810125350952), ('from', 0.9997801780700684), ('australian', 0.9997799396514893), ('police', 0.9997748732566833), ('if', 0.9997745752334595), ('a', 0.9997738003730774), ('told', 0.9997730255126953), ('his', 0.9997712969779968), ('forces', 0.9997707605361938), ('it', 0.9997703433036804)]
[('up', 0.9999563694000244), ('last', 0.9999563097953796), ('had', 0.9999523162841797), ('over', 0.9999522566795349), ('on', 0.9999514818191528), ('and', 0.9999509453773499), ('are', 0.9999508261680603), ('also', 0.9999488592147827), ('an', 0.9999486207962036), ('with', 0.9999478459358215)]
[('two', 0.9999306797981262), ('over', 0.9999291896820068), ('three', 0.9999275803565979), ('on', 

Now with caching

In [50]:
start = time.time()
for word in words:
    if 'voted' in most_similars_precalc:
        result = most_similars_precalc[word]
        print(result)
    else:
        result = model.wv.most_similar(word)
        most_similars_precalc[word] = result
        print(result)
    
end = time.time()
print(end-start)

[('action', 0.9987000226974487), ('would', 0.9986580610275269), ('could', 0.9986568093299866), ('need', 0.9986546635627747), ('will', 0.9986473917961121), ('it', 0.9986432790756226), ('legal', 0.9986417293548584), ('expected', 0.9986345171928406), ('there', 0.9986207485198975), ('we', 0.998618483543396)]
[('which', 0.9997810125350952), ('from', 0.9997801780700684), ('australian', 0.9997799396514893), ('police', 0.9997748732566833), ('if', 0.9997745752334595), ('a', 0.9997738003730774), ('told', 0.9997730255126953), ('his', 0.9997712969779968), ('forces', 0.9997707605361938), ('it', 0.9997703433036804)]
[('up', 0.9999563694000244), ('last', 0.9999563097953796), ('had', 0.9999523162841797), ('over', 0.9999522566795349), ('on', 0.9999514818191528), ('and', 0.9999509453773499), ('are', 0.9999508261680603), ('also', 0.9999488592147827), ('an', 0.9999486207962036), ('with', 0.9999478459358215)]
[('two', 0.9999306797981262), ('over', 0.9999291896820068), ('three', 0.9999275803565979), ('on', 

Clearly you can see the improvement but this difference will be even larger when we take more words in the consideration.

# Visualising the Word Embeddings

The word embeddings made by the model can be visualised by reducing dimensionality of the words to **2 dimensions using tSNE**.  
Visualisations can be used to notice semantic and syntactic trends in the data.  
  
Example: Semantic- words like cat, dog, cow, etc. have a tendency to lie close by  
         Syntactic- words like run, running or cut, cutting lie close together.  
Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.  
  
The function below can be used to plot the embeddings in an ipython notebook.  
It requires the model as the necessary parameter. If you don't have the model, you can load it by  

In [59]:
from sklearn.decomposition import IncrementalPCA 
from sklearn.manifold import TSNE    
import numpy as np                
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

def reduce_dimensions(model, plot_in_notebook = True):

    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []        # positions in vector space
    labels = []         # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    logging.info('starting tSNE dimensionality reduction. This may take some time.')
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
        
    # Create a trace
    trace = go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='text',
        text=labels
        )
    
    data = [trace]
    
    logging.info('All done. Plotting.')
    
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data)
    else:
        plot(data, filename='word-embedding-plot.html')

In [60]:
%matplotlib inline

In [61]:
%time reduce_dimensions(model)

CPU times: user 17.9 s, sys: 2.29 s, total: 20.2 s
Wall time: 20.1 s


## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).